# Dependencies

In [2]:
!pip install azureml

  Using cached https://files.pythonhosted.org/packages/ab/e8/76cd2cb6784b9039affd2c659eed1b3f46baf2e6b87a10b072a20b5b0113/azureml-0.2.7-py2.py3-none-any.whl


In [4]:
!pip install azureml.core

  Using cached https://files.pythonhosted.org/packages/82/59/cb226752e20d83598d7fdcabd7819570b0329a61db07cfbdd21b2ef546e3/SecretStorage-3.1.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3e/93/02056aca45162f9fc275d1eaad12a2a07ef92375afb48eabddc4134b8315/azure_graphrbac-0.61.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/85/60/370352f7ef6aa96c52fb001831622f50f923c1d575427d021b8ab3311236/contextlib2-0.6.0.post1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cc/ca/699d4754a932787ef353a157ada74efd1ceb6d1fc0bfb7989ae1e7b33111/docker-4.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/17/4724694ddb3311955ddc367eddcd0928f8ee2c7b12d5a6f0b12bca0b03db/azure_mgmt_authorization-0.60.0-py2.py3-none-any.whl
Processing c:\users\dariusb\appdata\local\pip\cache\wheels\62\b8\e1\e2719465b5947c40cd85d613d6cb33449b86a1ca5a6c574269\pathspec-0.6.0-cp37-none-any.whl
  Using cached https://fi

In [17]:
!pip install azureml.pipeline

  Using cached https://files.pythonhosted.org/packages/a1/53/234c53004f71f0717d8acd37876e0b65c121181167057b9ce1b1795f96a0/applicationinsights-0.11.9-py2.py3-none-any.whl


In [22]:
!pip install azureml.widgets

# Pipeline setup

In [50]:
from azureml.core import Workspace, Datastore

ws = Workspace.create(
   name = 'hackingjunction',
   subscription_id = 'a59b43cd-1c53-4a7e-b715-19b07f028dcf',
   resource_group = 'hacking',
   location = 'WestEurope',
   exist_ok = True)

In [ ]:
account_key = input()

In [44]:
storage_account = "hackingjunctio1306266651"
container_name = "inputs"

In [45]:
from azureml.core.runconfig import CondaDependencies, RunConfiguration

cd = CondaDependencies.create(pip_packages =['azureml-defaults', 'pandas', 'numpy', 'scipy', 'sklearn','azure-storage-blob==2.1.0'])

run_config = RunConfiguration(conda_dependencies = cd)
run_config.environment.docker.enabled = True
run_config.environment.spark.precache_packages = False

In [46]:
def_blob_store = Datastore(ws, 'workspaceblobstore')

In [47]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps.python_script_step import PythonScriptStep
import os
from azureml.core.compute import ComputeTarget

target = ws.compute_targets["JunctionCompute"]

input_ref = PipelineData(
    f"input",
    datastore = def_blob_store,
    output_name=f"input")

output_ref = PipelineData(
    f"output",
    datastore = def_blob_store,
    output_name=f"output")

train_step = PythonScriptStep(
    script_name = 'train.py',
    arguments = ['--input_path', input_ref,
                 '--output', output_ref,            
                 '--input_container', container_name,
                 '--output_container', 'outputs',
                 '--input_account', storage_account,
                 '--input_key', account_key],
    outputs = [output_ref, input_ref],
    compute_target = target,
    source_directory = './BehavioralAnalysis',
    allow_reuse = False,
    runconfig = run_config
)

pipeline_steps = [train_step]

In [48]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace = ws, steps = pipeline_steps)

In [49]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

pipeline_run = Experiment(ws, 'Behavioral_Analysis_Experiment').submit(pipeline)

#pipeline_run.wait_for_completion(show_output = True)
RunDetails(pipeline_run).show()

Created step train.py [9f32f366][7f91bf11-b09c-4272-bde7-e71c1468a6d9], (This step will run and generate new outputs)
Submitted PipelineRun 63d7c4c8-485c-445b-abeb-d5099b38b447
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/Behavioral_Analysis_Experiment/runs/63d7c4c8-485c-445b-abeb-d5099b38b447?wsid=/subscriptions/a59b43cd-1c53-4a7e-b715-19b07f028dcf/resourcegroups/hacking/workspaces/hackingjunction


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

# Deployment

In [ ]:
from azureml.pipeline.core import PublishedPipeline

pipelines = list(filter(lambda x: x._name == 'Behavioral_Analysis_Experiment', PublishedPipeline.list(ws)))
for pipeline in pipelines:
    pipeline.disable()

In [ ]:
pipeline = Pipeline(workspace = ws, steps = pipeline_steps)

published_pipeline = pipeline.publish(
    name="Behavioral_Analysis_Experiment",
    description="Retrains the model & computes behavioral tags for users",
    version=0.1)